<a href="https://colab.research.google.com/github/fbeilstein/topological_data_analysis/blob/master/persistent_homology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Main Engine (RUN)

import sympy as sp
import numpy as np
from sympy.matrices.normalforms import smith_normal_form
from sympy import Matrix, ZZ, pprint

def set_triangles(tt):
    triangles = [''.join(sorted(v)) for v in tt]
    print(f'triangles :  {triangles}')
    return triangles


def get_complex(tt):
    parts = []
    for t in tt:
        parts += [t, t[0], t[1], t[2], ''.join(sorted([t[0],t[1]])),
                  ''.join(sorted([t[1],t[2]])), ''.join(sorted([t[0],t[2]]))]
    complex_ = sorted(list(set(parts)))
    print(f'complex : {complex_}')
    return complex_


def get_chain_order(complex, order):
    chain = [c for c in complex if len(c)==order+1]
    print(f'{order}-chain : {chain}')
    return chain


def get_span(tt):
    string = ""
    for i,t in enumerate(tt):
        string  += f'{"-" if t[0]=="-" else "+"}x{i}*{t}'
    print("span    :", str(sp.simplify(string)))
    return tt


def calculate_boundary(tt, space="ker"):
    matrix, n = {}, len(tt)
    for i,t in enumerate(tt):
        for j,_ in enumerate(t):
            x = t[:j]+t[j+1:]
            if x=='' : continue
            if not (x in matrix):
                matrix[x] = n*[0]
            matrix[x][i] += (-1)**j
    show_expressions(matrix)
    calculate_ranks(matrix, n, space)


def show_expressions(matrix):
   expr = ""
   for var, coeffs in matrix.items():
        expr += f'+{var}*('
        for i,c in enumerate(coeffs):
          expr += f'{"+" if c>=0 else ""}{c}*x{i}'
        expr += ')'
   print("lin. operator:", str(sp.simplify(expr)) if expr else "0")


def calculate_ranks(matrix, n, space):
    to_np = [*matrix.values()]
    s = smith_normal_form(Matrix(to_np), domain=ZZ)
    diag = []
    for i in range(min(s.shape[0], s.shape[1])):
      diag.append(s[i,i])
    #pprint(s)
    print("Diagonal of Smith normal form :", diag)
    rank  = np.linalg.matrix_rank(np.matrix(to_np)) if to_np else 0
    print(f'ker     : {n-rank}' if space=="ker" else f'img     : {rank}')

In [ ]:
#@title Demo Homology

import IPython
from google.colab import output

def invoke(triangles):
  #output.clear()
  T = set_triangles(triangles)
  order = 0
  print()

  chain2 = get_chain_order(get_complex(T), order+1)
  for_image = get_span(chain2)
  calculate_boundary(for_image, "img")
  print()

  chain1 = get_chain_order(get_complex(T), order)
  for_kernel = get_span(chain1)
  calculate_boundary(for_kernel, "ker")
  return IPython.display.JSON({'hello': 'world',
                               'world': 'hello'})


main_str = '''
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta http-equiv="X-UA-Compatible" content="IE-edge"/>
<meta name="viewport" content="width=device-width, initial-scale=1.0"/>
<link   rel="stylesheet" href="https://fbeilstein.github.io/topological_data_analysis/js_common/styles.css"/>

<title> Document </title>
</head>
<body>
    <div class="container">
        <canvas id="canvas"></canvas>
    </div>
    <button id="button">GET TRIANGLES</button>
    <script src="https://fbeilstein.github.io/topological_data_analysis/js_common/script.js"></script>
</body>
</html>
'''

output.register_callback('notebook.homologies', invoke)
display(IPython.display.HTML(main_str))